<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Setup---Debug" data-toc-modified-id="Setup---Debug-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup - Debug</a></span></li><li><span><a href="#Setup---Imports" data-toc-modified-id="Setup---Imports-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setup - Imports</a></span></li><li><span><a href="#Setup---logging" data-toc-modified-id="Setup---logging-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Setup - logging</a></span></li><li><span><a href="#Setup---virtualenv-jupyter-kernel" data-toc-modified-id="Setup---virtualenv-jupyter-kernel-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Setup - virtualenv jupyter kernel</a></span></li><li><span><a href="#Setup---Initialize-Django" data-toc-modified-id="Setup---Initialize-Django-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Setup - Initialize Django</a></span></li></ul></li><li><span><a href="#Find-articles-to-be-coded" data-toc-modified-id="Find-articles-to-be-coded-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Find articles to be coded</a></span><ul class="toc-item"><li><span><a href="#which-articles-have-already-been-coded?" data-toc-modified-id="which-articles-have-already-been-coded?-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>which articles have already been coded?</a></span><ul class="toc-item"><li><span><a href="#Tag-the-coded-articles" data-toc-modified-id="Tag-the-coded-articles-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Tag the coded articles</a></span></li><li><span><a href="#Profile-the-coded-articles" data-toc-modified-id="Profile-the-coded-articles-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Profile the coded articles</a></span></li></ul></li><li><span><a href="#tag-all-local-news" data-toc-modified-id="tag-all-local-news-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>tag all local news</a></span><ul class="toc-item"><li><span><a href="#TODO" data-toc-modified-id="TODO-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>TODO</a></span><ul class="toc-item"><li><span><a href="#DONE" data-toc-modified-id="DONE-3.2.1.1"><span class="toc-item-num">3.2.1.1&nbsp;&nbsp;</span>DONE</a></span></li></ul></li><li><span><a href="#Grand-Rapids-Press-local-news" data-toc-modified-id="Grand-Rapids-Press-local-news-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Grand Rapids Press local news</a></span></li><li><span><a href="#Detroit-News-local-news" data-toc-modified-id="Detroit-News-local-news-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Detroit News local news</a></span></li></ul></li></ul></li><li><span><a href="#Code-Articles" data-toc-modified-id="Code-Articles-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Code Articles</a></span></li><li><span><a href="#TODO" data-toc-modified-id="TODO-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>TODO</a></span></li></ul></div>

# Introduction

- Back to [Table of Contents](#Table-of-Contents)

This is a notebook that expands on the OpenCalais code in the file `article_coding.py`, also in this folder.  It includes more sections on selecting publications you want to submit to OpenCalais as an example.  It is intended to be copied and re-used.

# Setup

- Back to [Table of Contents](#Table-of-Contents)

## Setup - Debug

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
debug_flag = False

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
import datetime
from django.db.models import Avg, Max, Min
import logging
import six

## Setup - logging

- Back to [Table of Contents](#Table-of-Contents)

configure logging for this notebook's kernel (If you do not run this cell, you'll get the django application's logging configuration.

In [ ]:
logging.basicConfig(
    level = logging.DEBUG,
    format = '%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    filename = '/home/jonathanmorgan/logs/django-research.log',
    filemode = 'w' # set to 'a' if you want to append, rather than overwrite each time.
)

## Setup - virtualenv jupyter kernel

- Back to [Table of Contents](#Table-of-Contents)

If you are using a virtualenv, make sure that you:

- have installed your virtualenv as a kernel.
- choose the kernel for your virtualenv as the kernel for your notebook (Kernel --> Change kernel).

Since I use a virtualenv, need to get that activated somehow inside this notebook.  One option is to run `../dev/wsgi.py` in this notebook, to configure the python environment manually as if you had activated the `sourcenet` virtualenv.  To do this, you'd make a code cell that contains:

    %run ../dev/wsgi.py
    
This is sketchy, however, because of the changes it makes to your Python environment within the context of whatever your current kernel is.  I'd worry about collisions with the actual Python 3 kernel.  Better, one can install their virtualenv as a separate kernel.  Steps:

- activate your virtualenv:

        workon research

- in your virtualenv, install the package `ipykernel`.

        pip install ipykernel

- use the ipykernel python program to install the current environment as a kernel:

        python -m ipykernel install --user --name <env_name> --display-name "<display_name>"
        
    `sourcenet` example:
    
        python -m ipykernel install --user --name sourcenet --display-name "research (Python 3)"
        
More details: [http://ipython.readthedocs.io/en/stable/install/kernel_install.html](http://ipython.readthedocs.io/en/stable/install/kernel_install.html)

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [3]:
# init django
django_init_folder = "/home/jonathanmorgan/work/django/research/work/phd_work"
django_init_path = "django_init.py"
if( ( django_init_folder is not None ) and ( django_init_folder != "" ) ):
    
    # add folder to front of path.
    django_init_path = "{}/{}".format( django_init_folder, django_init_path )
    
#-- END check to see if django_init folder. --#

In [4]:
%run $django_init_path

django initialized at 2019-08-01 02:27:25.965530


In [5]:
# context_text imports
from context_text.article_coding.article_coding import ArticleCoder
from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder
from context_text.collectors.newsbank.newspapers.GRPB import GRPB
from context_text.collectors.newsbank.newspapers.DTNB import DTNB
from context_text.models import Article
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase

# Find articles to be coded

- Back to [Table of Contents](#Table-of-Contents)

Tag all locally implemented hard news articles in database and all that have already been coded using Open Calais V2, then work through using OpenCalais to code all local hard news that hasn't alredy been coded, starting with those proximal to the coding sample for methods paper.

## which articles have already been coded?

- Back to [Table of Contents](#Table-of-Contents)

More precisely, find all articles that have Article_Data coded by the automated coder with type "OpenCalais_REST_API_v2" and tag the articles as "coded-open_calais_v2" or something like that.

Then, for articles without that tag, use our criteria for local hard news to filter out and tag publications in the year before and after the month used to evaluate the automated coder, in both the Grand Rapids Press and the Detroit News, so I can look at longer time frames, then code all articles currently in database.

Eventually, then, we'll code and examine before and after layoffs.

In [7]:
# look for publications that have article data:
# - coded by automated coder
# - with coder type of "OpenCalais_REST_API_v2"

# get automated coder
automated_coder_user = ArticleCoder.get_automated_coding_user()

print( "{} - Loaded automated user: {}, id = {}".format( datetime.datetime.now(), automated_coder_user, automated_coder_user.id ) )

2019-07-31 19:39:36.556744 - Loaded automated user: automated, id = 2


In [8]:
# try aggregates
article_qs = Article.objects.all()
pub_date_info = article_qs.aggregate( Max( 'pub_date' ), Min( 'pub_date' ) )
print( pub_date_info )

{'pub_date__max': datetime.date(2010, 11, 30), 'pub_date__min': datetime.date(2005, 1, 1)}


In [9]:
# find articles with Article_Data created by the automated user...
article_qs = Article.objects.filter( article_data__coder = automated_coder_user )

# ...and specifically coded using OpenCalais V2...
article_qs = article_qs.filter( article_data__coder_type = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION )

# ...and finally, we just want the distinct articles by ID.
article_qs = article_qs.order_by( "id" ).distinct( "id" )

# count?
article_count = article_qs.count()
print( "Found {} articles".format( article_count ) )

Found 579 articles


### Tag the coded articles

- Back to [Table of Contents](#Table-of-Contents)

Removing duplicates present from joining with Article_Data yields 579 articles that were coded by the automated coder.

Tag all the coded articles with `OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME`.

In [10]:
# declare variables
current_article = None
tag_name_list = None
article_count = None
untagged_count = None
already_tagged_count = None
newly_tagged_count = None
count_sum = None
do_add_tag = False

# init
do_add_tag = True

# get article_count
article_count = article_qs.count()

# loop over articles.
untagged_count = 0
already_tagged_count = 0
newly_tagged_count = 0
for current_article in article_qs:
    
    # get list of tags for this publication
    tag_name_list = current_article.tags.names()
    
    # is the coded tag in the list?
    if ( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME not in tag_name_list ):
        
        # are we adding tag?
        if ( do_add_tag == True ):

            # add tag.
            current_article.tags.add( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME )
            newly_tagged_count += 1
            
        else:

            # for now, increment untagged count
            untagged_count += 1
            
        #-- END check to see if we are adding tag. --#
        
    else:
        
        # already tagged
        already_tagged_count += 1
        
    #-- END check to see if coded tag is set --#
    
#-- END loop over articles. --#

print( "Article counts:" )
print( "- total articles: {}".format( article_count ) )
print( "- untagged articles: {}".format( untagged_count ) )
print( "- already tagged: {}".format( already_tagged_count ) )
print( "- newly tagged: {}".format( newly_tagged_count ) )
count_sum = untagged_count + already_tagged_count + newly_tagged_count
print( "- count sum: {}".format( count_sum ) )

Article counts:
- total articles: 579
- untagged articles: 0
- already tagged: 579
- newly tagged: 0
- count sum: 579


### Profile the coded articles

- Back to [Table of Contents](#Table-of-Contents)

Look at range of pub dates.

In [6]:
tags_in_list = []
tags_in_list.append( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME )
article_qs = Article.objects.filter( tags__name__in = tags_in_list )
print( "Matching article count: {}".format( article_qs.count() ) )

Matching article count: 589


- Original: 579
- after coding 10: 589 (tag is being set correctly bu Open Calais V2 coder)

In [12]:
# profile these publications
min_pub_date = None
max_pub_date = None
current_pub_date = None
pub_date_count = None
date_to_count_map = {}
date_to_articles_map = {}
pub_date_article_dict = None

# try aggregates
pub_date_info = article_qs.aggregate( Max( 'pub_date' ), Min( 'pub_date' ) )
print( pub_date_info )

# counts of pubs by date
for current_article in article_qs:
    
    # get pub_date
    current_pub_date = current_article.pub_date
    current_article_id = current_article.id
    
    # get count, increment, and store.
    pub_date_count = date_to_count_map.get( current_pub_date, 0 )
    pub_date_count += 1
    date_to_count_map[ current_pub_date ] = pub_date_count
    
    # also, store up ids and instances
    
    # get dict of article ids to article instances for date
    pub_date_article_dict = date_to_articles_map.get( current_pub_date, {} )
    
    # article already there?
    if ( current_article_id not in pub_date_article_dict ):
        
        # no - add it.
        pub_date_article_dict[ current_article_id ] = current_article
        
    #-- END check to see if article already there.
    
    # put dict back.
    date_to_articles_map[ current_pub_date ] = pub_date_article_dict
    
#-- END loop over articles. --#

# output dates and counts.

# get list of keys from map
keys_list = list( six.viewkeys( date_to_count_map ) )
keys_list.sort()
for current_pub_date in keys_list:
    
    # get count
    pub_date_count = date_to_count_map.get( current_pub_date, 0 )
    print( "- {} ( {} ) count: {}".format( current_pub_date, type( current_pub_date ), pub_date_count ) )
    
#-- END loop over dates --#

{'pub_date__max': datetime.date(2010, 7, 31), 'pub_date__min': datetime.date(2005, 1, 7)}
- 2005-01-07 ( <class 'datetime.date'> ) count: 1
- 2005-01-21 ( <class 'datetime.date'> ) count: 1
- 2005-02-07 ( <class 'datetime.date'> ) count: 1
- 2005-02-10 ( <class 'datetime.date'> ) count: 1
- 2005-05-10 ( <class 'datetime.date'> ) count: 1
- 2005-05-16 ( <class 'datetime.date'> ) count: 1
- 2005-06-06 ( <class 'datetime.date'> ) count: 1
- 2005-06-22 ( <class 'datetime.date'> ) count: 1
- 2005-07-02 ( <class 'datetime.date'> ) count: 1
- 2005-07-05 ( <class 'datetime.date'> ) count: 2
- 2005-07-07 ( <class 'datetime.date'> ) count: 1
- 2005-08-22 ( <class 'datetime.date'> ) count: 1
- 2005-09-04 ( <class 'datetime.date'> ) count: 1
- 2005-09-11 ( <class 'datetime.date'> ) count: 1
- 2005-09-25 ( <class 'datetime.date'> ) count: 1
- 2005-10-04 ( <class 'datetime.date'> ) count: 1
- 2005-10-20 ( <class 'datetime.date'> ) count: 1
- 2005-10-21 ( <class 'datetime.date'> ) count: 1
- 2005-10-

In [13]:
# look at the 2010-07-31 date
pub_date = datetime.datetime.strptime( "2010-07-31", "%Y-%m-%d" ).date()
articles_for_date = date_to_articles_map.get( pub_date, {} )
print( articles_for_date )

# get the article and look at its tags.
article_instance = articles_for_date.get( 6065 )
print( article_instance.tags.all() )

# loop over associated Article_Data instances.
for article_data in article_instance.article_data_set.all():
    
    print( article_data )
    
#-- END loop over associated Article_Data instances --#

{6065: <Article: 6065 - Jul 31, 2010, City and Region ( A6 ), UID: 1315C0760F2D0668 - Local ArtPeers registration opens ( Grand Rapids Press, The )>}
<QuerySet [<Tag: prelim_reliability_test>, <Tag: prelim_reliability_combined>, <Tag: coded-OpenCalaisV2ArticleCoder>]>
2180 - minnesota1 - no coder_type -- Article: 6065 - Jul 31, 2010, City and Region ( A6 ), UID: 1315C0760F2D0668 - Local ArtPeers registration opens ( Grand Rapids Press, The )
2200 - minnesota2 - no coder_type -- Article: 6065 - Jul 31, 2010, City and Region ( A6 ), UID: 1315C0760F2D0668 - Local ArtPeers registration opens ( Grand Rapids Press, The )
2281 - minnesota3 - no coder_type -- Article: 6065 - Jul 31, 2010, City and Region ( A6 ), UID: 1315C0760F2D0668 - Local ArtPeers registration opens ( Grand Rapids Press, The )
2969 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 6065 - Jul 31, 2010, City and Region ( A6 ), UID: 1315C0760F2D0668 - Local ArtPeers registration opens ( Grand Rapids Press, The )


## tag all local news

- Back to [Table of Contents](#Table-of-Contents)

Definition of local hard news by in-house implementor for Grand Rapids Press and Detroit News follow.  For each, tag all articles in database that match as "local_hard_news".

### TODO

- Back to [Table of Contents](#Table-of-Contents)

TODO:

- make class for GRPB at NewsBank.

    - also, pull the things that are newspaper specific out of ArticleCoder.py and into the GRPB.py class.

- refine "local news" and "locally created" regular expressions for Grand Rapids Press based on contents of `author_string` and `author_affiliation`.
- do the same for TDN.
- then, use the updated classes and definitions below to flag all local hard news in database for each publication.

#### DONE

- Back to [Table of Contents](#Table-of-Contents)

DONE:

- abstract out shared stuff from GRPB.py and DTNB.py into abstract parent class context_text/collectors/newsbank/newspapers/newsbank_newspaper.py

    - update DTNB.py to use the parent class.
    
- make class for GRPB at NewsBank.

    - context_text/collectors/newsbank/newspapers/GRPB.py

### Grand Rapids Press local news

- Back to [Table of Contents](#Table-of-Contents)

Grand Rapids Press local hard news:

- `context_text/examples/articles/articles-GRP-local_news.py`
- local hard news sections (stored in `Article.GRP_NEWS_SECTION_NAME_LIST`):

    - "Business"
    - "City and Region"
    - "Front Page"
    - "Lakeshore"
    - "Religion"
    - "Special"
    - "State"

- in-house implementor (based on byline patterns, stored in `sourcenet.models.Article.Q_GRP_IN_HOUSE_AUTHOR`):

    - Byline ends in "/ THE GRAND RAPIDS PRESS", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *THE GRAND RAPIDS PRESS$'`

    - Byline ends in "/ PRESS * EDITOR", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *PRESS .* EDITOR$' )`

    - Byline ends in "/ GRAND RAPIDS PRESS * BUREAU", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *GRAND RAPIDS PRESS .* BUREAU$' )`

    - Byline ends in "/ SPECIAL TO THE PRESS", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *SPECIAL TO THE PRESS$' )`
        
- can also exclude columns (I will not):

        grp_article_qs = grp_article_qs.exclude( index_terms__icontains = "Column" )

Need to work to further refine this.

Looking at affiliation strings:

    SELECT author_affiliation, COUNT( author_affiliation ) as affiliation_count
    FROM context_text_article
    WHERE newspaper_id = 1
    GROUP BY author_affiliation
    ORDER BY COUNT( author_affiliation ) DESC;
    
And at author strings for collective bylines:

    SELECT author_string, COUNT( author_string ) as author_count
    FROM context_text_article
    WHERE newspaper_id = 1
    GROUP BY author_string
    ORDER BY COUNT( author_string ) DESC
    LIMIT 10;


In [16]:
# filter queryset to just locally created Grand Rapids Press (GRP) articles.
# imports
from context_text.models import Article
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase
from context_text.collectors.newsbank.newspapers.GRPB import GRPB

# declare variables - Grand Rapids Press
do_apply_tag = False
tag_to_apply = None
grp_local_news_sections = []
grp_newspaper = None
grp_article_qs = None
article_count = -1

# declare variables - filtering
include_opinion_columns = True
tags_in_list = []
tags_not_in_list = []
filter_out_prelim_tags = False
random_count = -1

# declare variables - make list of article IDs from QS.
article_id_list = []
article_counter = -1
current_article = None
article_tag_name_list = None
article_update_counter = -1

# ==> configure

# configure - size of random sample we want
#random_count = 60

# configure - also, apply tag?
do_apply_tag = True
tag_to_apply = ContextTextBase.TAG_LOCAL_HARD_NEWS

# set up "local, regional and state news" sections
grp_local_news_sections = GRPB.LOCAL_NEWS_SECTION_NAME_LIST

# Grand Rapids Press
# get newspaper instance for GRP.
grp_newspaper = Newspaper.objects.get( id = GRPB.NEWSPAPER_ID )

# start with all articles
#grp_article_qs = Article.objects.all()

# ==> filter to newspaper, local news section list, and in-house reporters.

# ----> manually

# now, need to find local news articles to test on.
#grp_article_qs = grp_article_qs.filter( newspaper = grp_newspaper )

# only the locally implemented sections
#grp_article_qs = grp_article_qs.filter( section__in = grp_local_news_sections )

# and, with an in-house author
#grp_article_qs = grp_article_qs.filter( Article.Q_GRP_IN_HOUSE_AUTHOR )

#print( "manual filter count: {}".format( grp_article_qs.count() ) )

# ----> using Article.filter_articles()
grp_article_qs = Article.filter_articles( qs_IN = grp_article_qs,
                                          newspaper = grp_newspaper,
                                          section_name_list = grp_local_news_sections,
                                          custom_article_q = GRPB.Q_IN_HOUSE_AUTHOR )

print( "Article.filter_articles count: {}".format( grp_article_qs.count() ) )

# and include opinion columns?
if ( include_opinion_columns == False ):
    
    # do not include columns
    grp_article_qs = grp_article_qs.exclude( index_terms__icontains = "Column" )
    
#-- END check to see if we include columns. --#

'''
# filter to newspaper, section list, and in-house reporters.
grp_article_qs = Article.filter_articles( qs_IN = grp_article_qs,
                                          start_date = "2009-12-01",
                                          end_date = "2009-12-31",
                                          newspaper = grp_newspaper,
                                          section_name_list = grp_local_news_sections,
                                          custom_article_q = Article.Q_GRP_IN_HOUSE_AUTHOR )
'''

# how many is that?
article_count = grp_article_qs.count()

print( "Article count before filtering on tags: " + str( article_count ) )

# ==> tags

# tags to exclude
tags_not_in_list = []

# Example: prelim-related tags
#tags_not_in_list.append( "prelim_reliability" )
#tags_not_in_list.append( "prelim_network" ]
#tags_not_in_list.append( "minnesota1-20160328" )
#tags_not_in_list.append( "minnesota2-20160328" )

# for later - exclude articles already coded.
#tags_not_in_list.append( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME )

# exclude any already tagged with tag_to_apply
tags_not_in_list.append( tag_to_apply )

if ( ( tags_not_in_list is not None ) and ( len( tags_not_in_list ) > 0 ) ):

    # exclude those in a list
    print( "filtering out articles with tags: " + str( tags_not_in_list ) )
    grp_article_qs = grp_article_qs.exclude( tags__name__in = tags_not_in_list )

#-- END check to see if we have a specific list of tags we want to exclude --#

# include only those with certain tags.
tags_in_list = []

# Examples

# Examples: prelim-related tags
#tags_in_list.append( "prelim_unit_test_001" )
#tags_in_list.append( "prelim_unit_test_002" )
#tags_in_list.append( "prelim_unit_test_003" )
#tags_in_list.append( "prelim_unit_test_004" )
#tags_in_list.append( "prelim_unit_test_005" )
#tags_in_list.append( "prelim_unit_test_006" )
#tags_in_list.append( "prelim_unit_test_007" )

# Example: grp_month
#tags_in_list.append( "grp_month" )

if ( ( tags_in_list is not None ) and ( len( tags_in_list ) > 0 ) ):

    # filter
    print( "filtering to just articles with tags: " + str( tags_in_list ) )
    grp_article_qs = grp_article_qs.filter( tags__name__in = tags_in_list )
    
#-- END check to see if we have a specific list of tags we want to include --#

# filter out "*prelim*" tags?
#filter_out_prelim_tags = True
if ( filter_out_prelim_tags == True ):

    # ifilter out all articles with any tag whose name contains "prelim".
    print( "filtering out articles with tags that contain \"prelim\"" )
    grp_article_qs = grp_article_qs.exclude( tags__name__icontains = "prelim" )
    
#-- END check to see if we filter out "prelim_*" tags --#

# how many is that?
article_count = grp_article_qs.count()

print( "Article count after tag filtering: " + str( article_count ) )

# do we want a random sample?
if ( random_count > 0 ):

    # to get random, order them by "?", then use slicing to retrieve requested
    #     number.
    grp_article_qs = grp_article_qs.order_by( "?" )[ : random_count ]
    
#-- END check to see if we want random sample --#

# this is a nice algorithm, also:
# - http://www.titov.net/2005/09/21/do-not-use-order-by-rand-or-how-to-get-random-rows-from-table/

# make ID list, tag articles if configured to.
article_id_list = []
article_counter = 0
article_update_counter = 0
for current_article in grp_article_qs:

    # increment article_counter
    article_counter += 1

    # add IDs to article_id_list
    article_id_list.append( str( current_article.id ) )
    
    # apply a tag while we are at it?
    if ( ( do_apply_tag == True ) and ( tag_to_apply is not None ) and ( tag_to_apply != "" ) ):
    
        # yes, please.  Tag already present?
        article_tag_name_list = current_article.tags.names()
        if ( tag_to_apply not in article_tag_name_list ):

            # Add tag.
            current_article.tags.add( tag_to_apply )
            
            # increment counter
            article_update_counter += 1
            
        #-- END check to see if tag already present. --#
        
    #-- END check to see if we apply tag. --#

    # output the tags.
    if ( debug_flag == True ):
        print( "- Tags for article " + str( current_article.id ) + " : " + str( current_article.tags.all() ) )
    #-- END DEBUG --#

#-- END loop over articles --#

# output the list.
print( "grp_article_qs count: {}".format( grp_article_qs.count() ) )
print( "Found " + str( article_counter ) + " articles ( " + str( article_count ) + " )." )
print( "- Updated {} articles to add tag {}.".format( article_update_counter, tag_to_apply ) )
if ( debug_flag == True ):
    print( "List of " + str( len( article_id_list ) ) + " local GRP staff article IDs: " + ", ".join( article_id_list ) )
#-- END DEBUG --#


Article.filter_articles count: 43816
Article count before filtering on tags: 43816
filtering out articles with tags: ['local_hard_news']
Article count after tag filtering: 0
grp_article_qs count: 0
Found 0 articles ( 0 ).
- Updated 0 articles to add tag local_hard_news.


### Detroit News local news

- Back to [Table of Contents](#Table-of-Contents)

Detroit News local news:

- `context_text/examples/articles/articles-TDN-local_news.py`
- local hard news sections (stored in `from context_text.collectors.newsbank.newspapers.DTNB import DTNB` - `DTNB.NEWS_SECTION_NAME_LIST`):

    - "Business"
    - "Metro"
    - "Nation" - because of auto industry stories

- in-house implementor (based on byline patterns, stored in `DTNB.Q_IN_HOUSE_AUTHOR`):

    - Byline ends in "/ The Detroit News", ignore case.

        - `Q( author_varchar__iregex = r'.*\s*/\s*the\s*detroit\s*news$' )`

    - Byline ends in "Special to The Detroit News", ignore case.

        - `Q( author_varchar__iregex = r'.*\s*/\s*special\s*to\s*the\s*detroit\s*news$' )`

    - Byline ends in "Detroit News * Bureau", ignore case.

        - `Q( author_varchar__iregex = r'.*\s*/\s*detroit\s*news\s*.*\s*bureau$' )`   

In [10]:
# filter queryset to just locally created Detroit News (TDN) articles.
# imports
from context_text.models import Article
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase
from context_text.collectors.newsbank.newspapers.DTNB import DTNB

# declare variables - Detroit News
do_apply_tag = False
tag_to_apply = None
tdn_local_news_sections = []
tdn_newspaper = None
tdn_article_qs = None
article_count = -1

# declare variables - filtering
include_opinion_columns = True
tags_in_list = []
tags_not_in_list = []
filter_out_prelim_tags = False
random_count = -1

# declare variables - make list of article IDs from QS.
article_id_list = []
article_counter = -1
current_article = None

# ==> configure

# configure - size of random sample we want
#random_count = 60

# configure - also, apply tag?
do_apply_tag = False
tag_to_apply = ContextTextBase.TAG_LOCAL_HARD_NEWS

# set up "local, regional and state news" sections
tdn_local_news_sections = DTNB.LOCAL_NEWS_SECTION_NAME_LIST

# Detroit News
# get newspaper instance for TDN.
tdn_newspaper = Newspaper.objects.get( id = DTNB.NEWSPAPER_ID )

# start with all articles
#tdn_article_qs = Article.objects.all()

# ==> filter to newspaper, local news section list, and in-house reporters.

# ----> manually

# now, need to find local news articles to test on.
#tdn_article_qs = tdn_article_qs.filter( newspaper = tdn_newspaper )

# only the locally implemented sections
#tdn_article_qs = tdn_article_qs.filter( section__in = tdn_local_news_sections )

# and, with an in-house author
#tdn_article_qs = tdn_article_qs.filter( DTNB.Q_IN_HOUSE_AUTHOR )

#print( "manual filter count: {}".format( tdn_article_qs.count() ) )

# ----> using Article.filter_articles()
tdn_article_qs = Article.filter_articles( qs_IN = tdn_article_qs,
                                          newspaper = tdn_newspaper,
                                          section_name_list = tdn_local_news_sections,
                                          custom_article_q = DTNB.Q_IN_HOUSE_AUTHOR )

print( "Article.filter_articles count: {}".format( tdn_article_qs.count() ) )

# and include opinion columns?
if ( include_opinion_columns == False ):
    
    # do not include columns
    tdn_article_qs = tdn_article_qs.exclude( author_string__in = DTNB.COLUMNIST_NAME_LIST )
    
#-- END check to see if we include columns. --#

'''
# filter to newspaper, section list, and in-house reporters.
tdn_article_qs = Article.filter_articles( qs_IN = tdn_article_qs,
                                          start_date = "2009-12-01",
                                          end_date = "2009-12-31",
                                          newspaper = tdn_newspaper,
                                          section_name_list = tdn_local_news_sections,
                                          custom_article_q = DTNB.Q_IN_HOUSE_AUTHOR )
'''

# how many is that?
article_count = tdn_article_qs.count()

print( "Article count before filtering on tags: " + str( article_count ) )

# ==> tags

# tags to exclude
#tags_not_in_list = [ "prelim_reliability", "prelim_network" ]
#tags_not_in_list = [ "minnesota1-20160328", "minnesota2-20160328", ]

# for later - exclude articles already coded.
#tags_not_in_list = [ OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME ]

tags_not_in_list = None
if ( ( tags_not_in_list is not None ) and ( len( tags_not_in_list ) > 0 ) ):

    # exclude those in a list
    print( "filtering out articles with tags: " + str( tags_not_in_list ) )
    tdn_article_qs = tdn_article_qs.exclude( tags__name__in = tags_not_in_list )

#-- END check to see if we have a specific list of tags we want to exclude --#

# include only those with certain tags.
#tags_in_list = [ "prelim_unit_test_001", "prelim_unit_test_002", "prelim_unit_test_003", "prelim_unit_test_004", "prelim_unit_test_005", "prelim_unit_test_006", "prelim_unit_test_007" ]
#tags_in_list = [ "tdn_month", ]
tags_in_list = None
if ( ( tags_in_list is not None ) and ( len( tags_in_list ) > 0 ) ):

    # filter
    print( "filtering to just articles with tags: " + str( tags_in_list ) )
    tdn_article_qs = tdn_article_qs.filter( tags__name__in = tags_in_list )
    
#-- END check to see if we have a specific list of tags we want to include --#

# filter out "*prelim*" tags?
#filter_out_prelim_tags = True
if ( filter_out_prelim_tags == True ):

    # ifilter out all articles with any tag whose name contains "prelim".
    print( "filtering out articles with tags that contain \"prelim\"" )
    tdn_article_qs = tdn_article_qs.exclude( tags__name__icontains = "prelim" )
    
#-- END check to see if we filter out "prelim_*" tags --#

# how many is that?
article_count = tdn_article_qs.count()

print( "Article count after tag filtering: " + str( article_count ) )

# do we want a random sample?
if ( random_count > 0 ):

    # to get random, order them by "?", then use slicing to retrieve requested
    #     number.
    tdn_article_qs = tdn_article_qs.order_by( "?" )[ : random_count ]
    
#-- END check to see if we want random sample --#

# this is a nice algorithm, also:
# - http://www.titov.net/2005/09/21/do-not-use-order-by-rand-or-how-to-get-random-rows-from-table/

# make ID list, tag articles if configured to.
article_id_list = []
article_counter = 0
for current_article in tdn_article_qs:

    # increment article_counter
    article_counter += 1

    # add IDs to article_id_list
    article_id_list.append( str( current_article.id ) )
    
    # apply a tag while we are at it?
    if ( ( do_apply_tag == True ) and ( tag_to_apply is not None ) and ( tag_to_apply != "" ) ):
    
        # yes, please.  Add tag.
        current_article.tags.add( tag_to_apply )
        
    #-- END check to see if we apply tag. --#

    # output the tags.
    if ( debug_flag == True ):
        print( "- Tags for article " + str( current_article.id ) + " : " + str( current_article.tags.all() ) )
    #-- END DEBUG --#

#-- END loop over articles --#

# output the list.
print( "tdn_article_qs count: {}".format( tdn_article_qs.count() ) )
print( "Found " + str( article_counter ) + " articles ( " + str( article_count ) + " )." )
if ( debug_flag == True ):
    print( "List of " + str( len( article_id_list ) ) + " local TDN staff article IDs: " + ", ".join( article_id_list ) )
#-- END DEBUG --#


Article.filter_articles count: 13070
Article count before filtering on tags: 13070
Article count after tag filtering: 13070
tdn_article_qs count: 13070
Found 13070 articles ( 13070 ).


# Code Articles

- Back to [Table of Contents](#Table-of-Contente)

Retrieve just publications that are tagged as being local hard news and that also are not tagged as having been coded by OpenCalaisV2.

In [ ]:
# declare variables

# declare variables - article filter parameters
start_pub_date = None # should be datetime instance
end_pub_date = None # should be datetime instance
tags_in_list = []
tags_not_in_list = []
paper_id_in_list = []
section_list = []
article_id_in_list = []
params = {}

# declare variables - processing
do_i_print_updates = True
my_article_coding = None
article_qs = None
article_count = -1
coding_status = ""
limit_to = -1
do_coding = True

# declare variables - results
success_count = -1
success_list = None
got_errors = False
error_count = -1
error_dictionary = None
error_article_id = -1
error_status_list = None
error_status = ""
error_status_counter = -1

# first, get a list of articles to code.

# ! Set param values.

# ==> start and end dates
#start_pub_date = "2009-12-06"
#end_pub_date = "2009-12-12"

# ==> tagged articles

# Examples:
#tag_in_list = "prelim_reliability"
#tag_in_list = "prelim_network"
#tag_in_list = "prelim_unit_test_007"
#tag_in_list = [ "prelim_reliability", "prelim_network" ]
#tag_in_list = [ "prelim_reliability_test" ] # 60 articles - Grand Rapids only.
#tag_in_list = [ "prelim_reliability_combined" ] # 87 articles, Grand Rapids and Detroit.
#tag_in_list = [ "prelim_training_001" ]
#tag_in_list = [ "grp_month" ]

# ----> include articles when these tags are present.
#tags_in_list = None
tags_in_list = []
tags_in_list.append( ContextTextBase.TAG_LOCAL_HARD_NEWS )

# ---> exclude articles when these tags are present.
#tags_not_in_list = None
tags_not_in_list = []
tags_not_in_list.append( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME )

# ==> IDs of newspapers to include.
#paper_id_in_list = "1"

# ==> names of sections to include.
#section_list = "Lakeshore,Front Page,City and Region,Business"

# ==> just limit to specific articles by ID.
article_id_in_list = []
#article_id_in_list = [ 360962 ]
#article_id_in_list = [ 28598 ]
#article_id_in_list = [ 21653, 21756 ]
#article_id_in_list = [ 90948 ]
#article_id_in_list = [ 21627, 21609, 21579 ]
#article_id_in_list = [ 48778 ]
#article_id_in_list = [ 6065 ]
#article_id_in_list = [ 221858 ]
#article_id_in_list = [ 23804, 22630 ]
#article_id_in_list = [ 23804 ]

# filter parameters
params[ ArticleCoding.PARAM_START_DATE ] = start_pub_date
params[ ArticleCoding.PARAM_END_DATE ] = end_pub_date
params[ ArticleCoding.PARAM_TAGS_IN_LIST ] = tags_in_list
params[ ArticleCoding.PARAM_TAGS_NOT_IN_LIST ] = tags_not_in_list
params[ ArticleCoding.PARAM_PUBLICATION_LIST ] = paper_id_in_list
params[ ArticleCoding.PARAM_SECTION_LIST ] = section_list
params[ ArticleCoding.PARAM_ARTICLE_ID_LIST ] = article_id_in_list

# set coder you want to use.

# OpenCalais REST API v.2
params[ ArticleCoding.PARAM_CODER_TYPE ] = ArticleCoding.ARTICLE_CODING_IMPL_OPEN_CALAIS_API_V2

# get instance of ArticleCoding
my_article_coding = ArticleCoding()
my_article_coding.do_print_updates = do_i_print_updates

# set params
my_article_coding.store_parameters( params )

print( "Query Parameters: {}".format( params ) )

# create query set - ArticleCoding does the filtering for you.
article_qs = my_article_coding.create_article_query_set()

print( "After my_article_coding.create_article_query_set(), count: {}".format( article_qs.count() ) )
if ( article_qs._result_cache is None ):
    
    print( "article_qs evaluated: NO ( {} )".format( article_qs._result_cache ) )
    
else:
    
    print( "article_qs evaluated: YES" )

#-- END check to see if _result_cache --#

# limit for an initial test?
limit_to = 4990
if ( ( limit_to is not None ) and ( isinstance( limit_to, int ) == True ) and ( limit_to > 0 ) ):

    # yes.
    article_qs = article_qs[ : limit_to ]

#-- END check to see if limit --#

# get article count
if ( isinstance( article_qs, list ) == True ):

    # list - call len()
    article_list = article_qs
    article_count = len( article_list )
    
else:

    # not a list - call count()
    article_count = article_qs.count()
    
#-- END figure out how to get count --#

print( "Matching article count: " + str( article_count ) )

# Do coding?
if ( do_coding == True ):

    print( "do_coding == True - it's on!" )

    # yes - make sure we have at least one article:
    if ( article_count > 0 ):

        # invoke the code_article_data( self, query_set_IN ) method.
        coding_status = my_article_coding.code_article_data( article_qs )
    
        # output status
        print( "\n\n==============================\n\nCoding status: \"" + coding_status + "\"" )
        
        # get success count
        success_count = my_article_coding.get_success_count()
        print( "\n\n====> Count of articles successfully processed: " + str( success_count ) )    
        
        # if successes, list out IDs.
        if ( success_count > 0 ):
        
            # there were successes.
            success_list = my_article_coding.get_success_list()
            print( "- list of successfully processed articles: " + str( success_list ) )
        
        #-- END check to see if successes. --#
        
        # got errors?
        got_errors = my_article_coding.has_errors()
        if ( got_errors == True ):
        
            # get error dictionary
            error_dictionary = my_article_coding.get_error_dictionary()
            
            # get error count
            error_count = len( error_dictionary )
            print( "\n\n====> Count of articles with errors: " + str( error_count ) )
            
            # loop...
            for error_article_id, error_status_list in six.iteritems( error_dictionary ):
            
                # output errors for this article.
                print( "- errors for article ID " + str( error_article_id ) + ":" )
                
                # loop over status messages.
                error_status_counter = 0
                for error_status in error_status_list:
                
                    # increment status
                    error_status_counter += 1

                    # print status
                    print( "----> status #" + str( error_status_counter ) + ": " + error_status )
                    
                #-- END loop over status messages. --#
            
            #-- END loop over articles. --#
   
        #-- END check to see if errors --#
    
    #-- END check to see if article count. --#
    
else:
    
    # output matching article count.
    print( "do_coding == False, so dry run" )
    
#-- END check to see if we do_coding --#


Query Parameters: {'start_date': None, 'end_date': None, 'tags_in_list_IN': ['local_hard_news'], 'tags_not_in_list_IN': ['coded-OpenCalaisV2ArticleCoder'], 'publications': [], 'section_list': [], 'article_id_list': [], 'coder_type': 'open_calais_api_v2'}
After my_article_coding.create_article_query_set(), count: 43254
article_qs evaluated: YES
Matching article count: 4990
do_coding == True - it's on!


==> article 1: 377049 - Frontline Community Church


==> article 2: 377011 - Soldier turned pastor uses modern weapons in - A war for souls


==> article 3: 377069 - Series explores religion, racial misconceptions - Racial assumptions can skew Scripture, a Hope professor says


==> article 4: 377088 - "The Fruitful Life"


==> article 5: 376913 - Church unites behind challenge of evangelism - "The Fruitful Life," a 40-day study of what it means to bear fruit as believers
in Christ



==> article 6: 376924 - Authors offer soaring tales of rare birds


==> article 7: 376946 - Relief worker



==> article 50: 377851 - Hope a long shot to win the MIAA title - Flying Dutchmen will improve, but not enough


==> article 51: 377672 - Hope trio coming into its own - Boles, Wood, Henderson help team win tourney


==> article 52: 377841 - Rough road - Entering MIAA play, hope trying to find way out of funk


==> article 53: 377954 - EGR native links Australian, U.S. firms - New trade agreement will benefit both countries, Christine Gibbs-Stewart says


==> article 54: 378102 - Deal with Swedish firm helps Inrad - Kentwood company expects to add jobs


==> article 55: 378202 - Riviera's report raises warning - Auditor doubts company can continue after bigger-than-expected loss


==> article 56: 378092 - Bargain hunting


==> article 57: 378007 - Estate sale lures collectors


==> article 58: 378049 - 'He just wanted to have a good time' - Police say Holland man was slain after an argument that started in a Grand Rapids bar


==> article 59: 378006 - Resolutions you can handle - For



==> article 99: 378293 - District to raze ice cream shop - Frosty Boy lot will become storage, parking for Creston, Palmer


==> article 100: 378416 - Mining project turned down


==> article 101: 378462 - Township sued over zoning - Restoration of residential status sought


==> article 102: 378251 - Howard Stern yanked - Shock jock dropped from local station, apparently over tendency to promote his future employer


==> article 103: 378375 - Focus shifts to finding killer - State police say 23-year-old Julia Dawson was murdered


==> article 104: 378430 - Howard Stern yanked - Shock jock dropped from local station, apparently over tendency to promote his future employer


==> article 105: 378504 - Focus shifts to finding killer - State police say 23-year-old Julia Dawson was murdered


==> article 106: 378536 - Tragedy can result if kids don't learn danger of fire - Experts say parents tend to ignore youngsters' fascination with flames


==> article 107: 378496 - Playing with fire




==> article 147: 379249 - Mentor brings FIRST to alternative high - The Nonconformants are registered to compete in two tournaments


==> article 148: 379223 - Competitive cheer


==> article 149: 378911 - Past stars shine again for Hope - Top women alumni face JV basketball team Saturday


==> article 150: 378809 - Panthers cheer team seeks return to top - Squad finished fifth at state after making top three previous four years


==> article 151: 378756 - Flying Dutchmen's fresh start - Hope men open MIAA play with convincing victory over Tri-State


==> article 152: 379023 - Family bowling tournament puts priority on fun - Event pairing child with adult bowlers will held during next two weekends


==> article 153: 378652 - It's time to bury 2004 catch phrases


==> article 154: 379374 - Shoppers gained steam - Gift cards were season's biggest hit


==> article 155: 379475 - Businesswoman buys into neighborhood revival - Antiques store becomes shopping destination


==> article 156:



==> article 195: 379903 - Learning to give, learning to accept


==> article 196: 379712 - Ministry uses culture to share faith


==> article 197: 379852 - Without answers, ask 'how' not 'why' - There often is meaning in response to tragedy


==> article 198: 379885 - Local faith groups join in relief effort


==> article 199: 379651 - Through teamwork, tragedies are overcome


==> article 200: 379793 - Editorial helps to get CRC readers' goats


==> article 201: 380062 - Our voices heard -- even in Spanish - State commission now includes four from area


==> article 202: 380433 - A step above - Wolverine set pace in good year for local stocks


==> article 203: 380378 - Auto Show's theme: anything (that) goes - GM unveiling three advanced hybrid vehicles


==> article 204: 380018 - 'Hot Dog Man' feeds need for GR's hungry - It's a humble menu. But, for those who line up early each week, it's a lifeline


==> article 205: 380071 - Lunch money


==> article 206: 380147 - School will t

...



==> article 2061: 408449 - Head of the class - Hope excels as host of Division III national swim meet


==> article 2062: 408564 - Firm picks area for expansion - Emerald Graphics will begin production in Kentwood, Cascade Township


==> article 2063: 408581 - Retail, apartments planned for vacant buildings


==> article 2064: 408752 - Firm picks area for expansion - Emerald Graphics will begin production in Kentwood, Cascade Township


==> article 2065: 408662 - Smoke signals - Company's ban on smokers gains acceptance, attorney says


==> article 2066: 408474 - Employers expect to hire, not lay off, this spring - Survey shows stronger hiring intentions between April and June


==> article 2067: 408685 - Alticor urges better workers' health


==> article 2068: 408670 - CEO says Delphi fiscally sound - Accounting woes not reason for his departure, Battenberg says


==> article 2069: 408538 - Kimball ready to talk cuts in budget - City manager will tell commissioners they need to trim



==> article 2106: 409258 - Minister's message: Be smart - He must live with consequences of earlier, carefree lifestyle


==> article 2107: 408846 - Up for bids


==> article 2108: 409044 - Communication problems slowed pileup rescue - Authorities say they would establish common frequencies for disasters


==> article 2109: 409130 - Police try to revive Child Watch program


==> article 2110: 409124 - Classroom pledge heads to House - Senate unanimously passes bill to allow for recitation in schools


==> article 2111: 409224 - Students, parents get fair look at colleges


==> article 2112: 409215 - Advocate for disabled admits to defrauding investors - 'I did not use the money as promised,' he says as he pleads guilty
to scam



==> article 2113: 408965 - Commissioners float ideas to plug budget shortfall - Grand Rapids faces an $11 million deficit, and some officials look to
Lansing for help



==> article 2114: 409118 - Cabbies to have drug tests when applying for licenses - New r



==> article 2152: 409759 - Baby girl was not sold, says Michigan woman - Lake County couple were taking legal steps for full guardianship of Kentucky
infant, lawyer says



==> article 2153: 409812 - Baby girl was not sold, says Michigan woman - Lake County couple were taking legal steps for full guardianship of Kentucky
infant, lawyer says



==> article 2154: 409979 - From schools to sports bars, we celebrate our inner Irish


==> article 2155: 409909 - Farmland sought for conservation effort - State program encourages improvements at farms to protect water quality


==> article 2156: 409649 - Developer tweaks size, price for Baker Lofts - Condominiums also will be for sale sooner


==> article 2157: 409575 - Big City Swing - Holland Christian group presents variety of musical styles at showcase


==> article 2158: 409945 - Driver to stand trial for girl's crash death - Defense says child died because she did not have seat belt


==> article 2159: 409764 - Church services at park i



==> article 2197: 410298 - Lords of the pool - Holland native helps Kenyon college lead swimming finals


==> article 2198: 410223 - Sprague hits milestone with NASCAR Trucks - Tonight he will be second driver to reach 200 races


==> article 2199: 410037 - Relay gives Panthers fast start at finals - Winning 200 medley was boost, since Chix won race at earlier swim meet


==> article 2200: 410076 - Church offers free help for tax time - Pastor says people with financial problems who seek help do not have to attend services


==> article 2201: 410110 - Closing off beaches hides God's gift - Walks along beach shows connections to nature


==> article 2202: 410331 - Competitive, but still too high - State insurance chief seeks legal tools to protect consumers


==> article 2203: 410356 - Village seeks payment from gas station owner - Middleville says property is contaminated


==> article 2204: 410236 - Allegan County won't limit length of board chairman's tenure - Proposal would have r



==> article 2243: 410756 - Diekema siblings wear record smiles after meet finishes - Jordan (Zeeland), Megan (Calvin) earn honors


==> article 2244: 410913 - Little time to celebrate for Heyboe - Holland Christian champ heads to series in Cleveland


==> article 2245: 410952 - A luge official on solid ground - Holland's Johnson back from officiating championships


==> article 2246: 411009 - Solid Steele, but she wanted more - Hope senior sought stronger final showing at D-III final


==> article 2247: 411085 - 'I've been dead for over 200 years' - Director brings classical composers to life


==> article 2248: 411008 - Democrats slam plans for Social Security - They critique Bush's call for adding private accounts to retirement program
ValueError parsing OpenCalais JSON for Article 411008 - raw response body: You exceeded the concurrent request limit for your license key. Please try again later or contact support to upgrade your license.
======> In code_article_data(): ERROR - Erro

- started: execution queued 22:42:01 2019-07-31


# TODO

- Back to [Table of Contents](#Table-of-Contents)

TODO:

- Save log of coding first 4990 of next round of data.
- for next round of coding, sort on publication date, descending, so we fill in the year before and after the layoffs first.
- start loading data from XML
- move data from Article_Data into context.
- make the network data creator work against context, then generalize it for tie and node types.
- adjust django logging to output DEBUG, then test Article.filter_articles() to see where QuerySet is evaluated (DISTINCT check?).
- think how we specify which class to use for author strings - needs to be speced to an interface, but not just a newsbank one - so, abstraction here should be higher up - in shared?
